In [4]:
#!/usr/bin/env python3
# filter_relabel_fasta.py
# Usage: python filter_relabel_fasta.py refs.csv input.fasta output.fasta

import sys
import csv
from Bio import SeqIO


In [61]:
csv_path = "/Volumes/Samsung_1TB/Zooplankton/Metagenomics/DBs/DB_wrangling/speciesIDs_withOutgroups.csv"
fasta_in  = "/Volumes/Samsung_1TB/Zooplankton/Metagenomics/DBs/BOLD_pull_direct/COIonly_BOLD_Public.03-Oct-2025.fasta"
fasta_out = "/Volumes/Samsung_1TB/Zooplankton/Metagenomics/DBs/BOLD_pull_direct/dada_addSpecies_withOutgroups.fasta"

In [54]:
id_col = "ID"
cols = ["Kingdom","Phylum","Class","Order","Family","Genus", "Species"]  # levels 1..6

In [55]:
map_header = {}
with open(csv_path, newline='') as fh:
    rdr = csv.DictReader(fh)
    for row in rdr:
        rid = row.get(id_col)
        if not rid:
            continue
        # build Level1;Level2;...;Level6; string (coalesce missing to empty)
        header = ";".join((row.get(c,"").strip() for c in cols)) + ";"
        map_header[rid.strip()] = header
    

In [62]:
# To build assign species file, build line differently

id_col = "ID"
cols = ["Species"]
map_header = {}

with open(csv_path, newline='') as fh:
    rdr = csv.DictReader(fh)
    for row in rdr:
        rid = row.get(id_col)
        if not rid:
            continue
        header = " ".join((row.get(c,"").strip() for c in cols))
        map_header[rid.strip()] = header

In [64]:
# Stream FASTA and write matches
with open(fasta_out, "w") as outfh:
    for rec in SeqIO.parse(fasta_in, "fasta"):
        # rec.description contains the full header line (without leading '>')
        # the first '|' field is the ID
        fasta_id = rec.description.split("|", 1)[0]
       # print(fasta_id)
        if fasta_id in map_header:
            rec.id = (fasta_id + " " + map_header[fasta_id]) # SeqIO will write >rec.id
            rec.description = ""             # avoid adding original desc
            SeqIO.write(rec, outfh, "fasta")

In [21]:
# To build assignSpecies file

'Animalia;Arthropoda;Malacostraca;Isopoda;Chaetiliidae;Glyptonotus;Glyptonotus antarcticus;'

In [63]:
len(map_header)

236741